In [53]:
import os
import json

In [55]:
# File path
working_dir = os.path.dirname(os.path.abspath(''))
data_process_dir = working_dir + "\\src\\data\\processing-batch\\data-process"
stop_word_file = working_dir + "\\src\\data\\processing-batch\\vietnamese-stopwords-dash.txt"
v_json_export_path = working_dir + "\\src\\data\\processing-batch\\topic_model.json"

In [7]:
extension = ".txt.pro"
delimiter_text = "/"
noun_type = "N"
topics = [
    "ThoiSu",
    "TheGioi",
    "KinhDoanh",
    "BatDongSan",
    "KhoaHoc",
    "GiaiTri",
    "TheThao",
    "PhapLuat",
    "GiaoDuc",
    "SucKhoe",
    "DoiSong",
    "DuLich",
    "SoHoa",
    "Xe",
]

In [25]:
# Lấy ra các hư từ
stop_word_file = open(stop_word_file, "r", encoding="utf-8")
stop_words = stop_word_file.read()
stop_words = stop_words.split("\n")
stop_words.append("ảnh")

TypeError: expected str, bytes or os.PathLike object, not TextIOWrapper

In [37]:
def get_noun_word(words: list):
    noun_words = []
    for text_split in words:
        text_split = text_split
        text_and_type = text_split.split(delimiter_text)
        type = text_and_type.pop()
        if type == noun_type:
            text = text_and_type.pop()
            # Nếu từ không phải hư từ thì đưa vào tập V
            if text.lower() not in stop_words:
                noun_words.append(text.lower())
    return noun_words

In [40]:
S = {f"{topic}":[] for topic in topics}
V = {f"{topic}":[] for topic in topics}
N = {}
n = {}

In [41]:
# Xử lý lấy tất cả danh từ đưa vào tập V
for topic in topics:
    data_process_file_path = data_process_dir + f"\\{topic}{extension}"
    with open(data_process_file_path, "r", encoding="utf-8") as file:
        text = file.read()
        # Xoá các ký tự xuống dòng
        text = text.replace("\n", " ")
        
        # Tách các từ
        text_splits = text.split(" ")
        
        # Tách câu
        sentences_splits = text.split("./.")
        
        # Đưa các từ thuộc danh từ vào tập V
        V[f'{topic}'] = get_noun_word(text_splits)
        
        # Đưa các câu vào tập S
        for sentences_split in sentences_splits:
            S[f'{topic}'].append(sentences_split)

In [42]:
# Tìm kiếm từ core cho mỗi chủ đề
for topic in topics:
    n = {}
    for text in V[f'{topic}']:
        if text in n.keys():
            n[f"{text}"] += 1
        else:
            n[f"{text}"] = 1
    
    N[f'{topic}'] = [max(n, key=n.get), n[max(n, key=n.get)]]

In [43]:
N

{'ThoiSu': ['xe', 639],
 'TheGioi': ['tổng_thống', 1587],
 'KinhDoanh': ['công_ty', 106],
 'BatDongSan': ['dự_án', 2283],
 'KhoaHoc': ['công_nghệ', 772],
 'GiaiTri': ['phim', 614],
 'TheThao': ['trận', 1149],
 'PhapLuat': ['công_ty', 741],
 'GiaoDuc': ['trường', 2268],
 'SucKhoe': ['bác_sĩ', 951],
 'DoiSong': ['nhân_viên', 50],
 'DuLich': ['du_khách', 1417],
 'SoHoa': ['công_ty', 613],
 'Xe': ['xe', 2992]}

In [51]:
threshhold = 0

# Cập nhật lại V cho các chủ đề
V = {f"{topic}":[] for topic in topics}
for topic in topics:
    n = {}
    
    # Lấy ra từ core
    core_word = N[f'{topic}']
    
    for sentence in S[f'{topic}']:
        # Tách các từ
        words = sentence.split(" ")
        
        
        # Lấy ra các danh từ
        noun_words = get_noun_word(words)
        
        # Nếu như core word có mặt trong câu thì cộng lại số lượng những danh từ trong đó
        if core_word[0] in noun_words:
            for noun_word in noun_words:
                if noun_word in n.keys():
                    n[f"{noun_word}"] += 1
                else:
                    n[f"{noun_word}"] = 1
        
    # Nếu trong 1 câu cùng xuất hiện từ chủ đề và 1 danh từ thì đưa danh từ vào tập chủ đề đó dựa trên threshold
    for word, frequency in n.items():
        if frequency/core_word[1] > threshhold:
            V[f'{topic}'].append([word.lower(), frequency])

In [61]:
# Export thành file json
# Serializing json
json_object = json.dumps(V, indent=4)
 
# Writing to sample.json
with open(v_json_export_path,"w" , encoding="utf-8") as outfile:
    json.dump(V, outfile)